# Clothing images classification using Tensorflow/Keras

Material adopted from [Tensorflow tutorials](https://www.tensorflow.org/tutorials/keras/classification)

In this workflow we train a neural network to classify images of clothing (sneakers and shirts) using the high level tensroflow keras API. The entire training process can be broken down into a sequence of steps described as follows

1. Import and generate the fashion dataset
2. Preprocess the datasets: Scale the values in the images to range between [0, 1]
3. Build and compile the model (Keras)
4. Cross validate the model on the training set
5. Optimize hyper-parameters (Regularization, Dropout rates ...)

In [ ]:
# Import tensorflow and tf.keras
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from covalent.executor import SlurmExecutor, SSHExecutor
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import ParameterSampler, cross_val_score
from tensorflow.keras.regularizers import l2

print(tf.__version__)

from typing import Callable, List

import covalent as ct

### Import and load the Fashion dataset'

* [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) contains 70,000 grayscale images in 10 different categories
* Each image is a 28 x 28 pixels in size
* Each category in the image is label by an integer between 0 and 9

| Label | Category |
| ------|----------|
| 0     | T-shirt/top |
| 1     | Trouser |
| 2     | Pullover |
| 3     | Dress |
| 4     | Coat |
| 5     | Sandal |
| 6     | Shirt |
| 7     | Sneaker |
| 8     | Bag |
| 9     | Ankle Boot |

In [ ]:
def load_dataset():
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    return (
        train_images[:10000],
        train_labels[:10000],
        test_images[:1000],
        test_labels[:1000],
    )


train_images, train_labels, test_images, test_labels = load_dataset()

### Preprocess the dataset

Scale all pixel values to range between [0 - 1]

Lets inspect a single image first

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
@ct.electron
def scale_dataset(train_images, test_images):
    return train_images / 255.0, test_images / 255.0

In [ ]:
scaled_train_images, scaled_test_images = scale_dataset(train_images, test_images)

In [ ]:
plt.figure()
plt.imshow(scaled_train_images[0], cmap=plt.cm.binary)
plt.colorbar()

### Build the Keras model

We use this electron to abstract building/compiling of the keras model given the three required input parameters

* `hidden_layer_width`
    * Integer describing the width of the single hidden layer
    
* `dropout_rate`
    * float between [0, 1] marking the dropout rate for the network
    
* `l2_penalty`
    * L2 regularization rate to prevent over-fitting

### Create the remote executors

In [ ]:
slurm_executor = SlurmExecutor(
    username="venkat",
    address="beehive.agnostiq.ai",
    remote_workdir="/federation/venkat/workdir",
    ssh_key_file="/home/venkat/.ssh/id_ed25519",
    poll_freq=10,
    conda_env="ieee",
    options={
        "partition": "debug",
        "ntasks": 1,
        "cpus-per-task": 2,
        "chdir": "/federation/venkat/workdir",
        "nodelist": "beehive-debug-st-t2medium-1",
    },
)


ssh_executor = SSHExecutor(
    username="venkat",
    hostname="testbed.balavk.net",
    remote_cache_dir="/home/venkat/workdir",
    ssh_key_file="/home/venkat/.ssh/id_ed25519",
)

In [ ]:
@ct.electron
def build_keras_classifier(
    hidden_layer_width: int,
    dropout_rate: float,
    l2_penalty: float,
    optimizer: str = "adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
):
    """Build a sequential keras model with two hidden layers with widths hw1 and hw2"""
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    model.add(
        tf.keras.layers.Dense(
            hidden_layer_width, activation="relu", kernel_regularizer=l2(l2_penalty)
        )
    )
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    model.add(
        tf.keras.layers.Dense(
            hidden_layer_width, activation="relu", kernel_regularizer=l2(l2_penalty)
        )
    )
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    model.add(tf.keras.layers.Dense(10))

    # Compile the model
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model

### Convert Keras model to a sklearn compatible format

In [ ]:
@ct.electron
def get_sklearn_classifier(build_fn):
    """Returns an sklearn compatible format of the Keras Classifier"""
    return KerasClassifier(build_fn=build_fn)

### Perform n-fold cross validation of the model

In [ ]:
#@ct.electron(executor=ssh_executor)
@ct.electron
def cross_validate(
    model,
    train_images,
    train_labels,
    n_folds: int = 3,
    model_fit_params={"epochs": 10, "batch_size": 128, "verbose": 0},
):
    scores = cross_val_score(
        model, train_images, train_labels, cv=n_folds, fit_params=model_fit_params
    )
    return scores

### Main cross validation workflow

In [ ]:
#@ct.electron(executor=ssh_executor)
@ct.electron
@ct.lattice
def classifier_cv_workflow(
    training_set,
    training_labels,
    hidden_layer_width: int,
    dropout_rate: float = 0.0,
    l2_penalty: float = 0.1,
):
    # Preprocess the dataset
    scaled_train_images, scaled_test_images = scale_dataset(train_images, test_images)

    # Build the keras model
    classifier = get_sklearn_classifier(
        build_keras_classifier(hidden_layer_width, dropout_rate, l2_penalty)
    )

    # cross validate classifier
    scores = cross_validate(classifier, scaled_train_images, train_labels, n_folds=3)

    return scores

### Dispatch a single iteration of the cross validation workflow

In [ ]:
train_images, train_labels, test_images, test_labels = load_dataset()
dispatch_id = ct.dispatch(classifier_cv_workflow)(
    train_images, train_labels, 10, 0.0, 0.1
)

### Get cv scores

In [ ]:
cv_scores = ct.get_result(dispatch_id, wait=True).result

The `classifier_cv_workflow` performs a 3 fold cross validation for a specific values for the parameters i.e.
* `hidden_layer_width`: 10
* `dropout_rate`: 0.0
* `l2_penalty`: 0.1

We can now embed the `classifier_cv_workflow` as a **sublattice** of a larger workflow that runs `classifier_cv_workflow` for a variety of hyper parameters. To this end, we create a new workflow `optimize_hyperparameters` that feeds randomly generated hyperparameter values into `classifier_cv_workflow`, gets the cross validates scores and saves the results in a dictionary. We use `sklearn.model_selection.ParameterSampler` to return parameter values from distributions

In [ ]:
from scipy.stats.distributions import randint, uniform

### Define the parameter space to be randomly sampled

In [ ]:
pgrid = {
    "dropout_rate": uniform(0.0, 0.1),
    "l2_penalty": uniform(0, 1),
    "hidden_layer_width": randint(10, 100),
}

### This electron return the average cross validation score return by `cross_val_score`

In [ ]:
@ct.electron
def get_avg_model_score(cv_scores: List[float]):
    return np.mean(cv_scores)

In [ ]:
@ct.lattice
def optimize_hyperparameters(train_images, train_labels, parameters):
    result = []
    for index, parameter in enumerate(parameters):
        cv_scores = classifier_cv_workflow(
            train_images,
            train_labels,
            hidden_layer_width=parameter["hidden_layer_width"],
            dropout_rate=parameter["dropout_rate"],
            l2_penalty=parameter["l2_penalty"],
        )
        result.append(
            {"parameters": parameter, "score": get_avg_model_score(cv_scores)}
        )
    return result

### Dispatch the hyperparameter optimization workflow

In [ ]:
random_sampled_parameters = list(ParameterSampler(pgrid, n_iter=1))

dispatch_id = ct.dispatch(optimize_hyperparameters)(
    train_images, train_labels, random_sampled_parameters
)

In [ ]:
cv_scores = ct.get_result(dispatch_id, wait=True).result

In [ ]:
cv_scores